# Financial Recovery Group

### Technical Assessment (Home Exercise) - Data Analyst

#### Interviewer: Qin Luo (Lead Data Engineer)

#### Author: Rishi Raj Dutta

##### Date: 10/10/2020

### Objective: Analyze the input excel file containing risk adjustment socres for members in the form of hierarchal conditional category (HCC) codes and summarize the members will all potential risk categories in order to estimate future health care cost for patients.

##### Importing python libraries that we will use for data preprocessing and modeling.

In [1]:
import numpy as np
import pandas as pd
import re

##### Reading the raw excel input file without any preprocssing.

In [2]:
raw_df = pd.read_excel('/Users/rishirajdutta/Documents/Job Hunt/FRG/archive/HCC_Transpose_Problem.xlsx')

In [3]:
raw_df

,Input Table:\nHCC_Detail,MemberID,ActivityYear,HCC01,HCC02,HCC03,HCC04,HCC05,HCC06,HCC07,HCC08,HCC09,HCC10
0,NaN,1001,2019,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1001,2020,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,NaN,1002,2019,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,NaN,1003,2020,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,1004,2020,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NaN,1005,2020,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Output Table:\nHCC_Summary,MemberID,ActivityYear,HCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1001,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1001,2020,"1,5,8",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Dropping the unwanted rows and columns (such as output table which came as a part of the raw excel file).

In [4]:
df = raw_df.iloc[0:6].drop(['Input Table:\nHCC_Detail'], axis = 1)
df

,MemberID,ActivityYear,HCC01,HCC02,HCC03,HCC04,HCC05,HCC06,HCC07,HCC08,HCC09,HCC10
0,1001,2019,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,2020,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1002,2019,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1003,2020,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1004,2020,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1005,2020,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


##### Prining the columns headers to see if everything looks as expected and identify an index for optimizing preprocessing tasks

In [5]:
df.columns

Index(['MemberID', 'ActivityYear', 'HCC01', 'HCC02', 'HCC03', 'HCC04', 'HCC05',
       'HCC06', 'HCC07', 'HCC08', 'HCC09', 'HCC10'],
      dtype='object')

##### Making the MemberID as the Index

In [6]:
df = df.set_index('MemberID')

##### Converting the values from float to integers for uniform data preprocessing & avoid future errors

In [7]:
df = df.astype(int)

In [8]:
df

,ActivityYear,HCC01,HCC02,HCC03,HCC04,HCC05,HCC06,HCC07,HCC08,HCC09,HCC10
MemberID,,,,,,,,,,,
1001,2019,0,0,0,0,0,0,0,0,0,0
1001,2020,1,0,0,0,1,0,0,1,0,0
1002,2019,0,1,0,0,0,0,1,0,0,0
1003,2020,0,0,0,0,0,0,0,0,0,0
1004,2020,1,0,0,0,0,0,0,0,0,0
1005,2020,0,0,1,0,0,1,0,0,0,0


##### Dropping the ActivityYear column to create a Matrix of 0's and 1's

In [9]:
hcc_df = df.iloc[:,1:]

##### Here first we are apply the "dot" operation to perform a dot product on the dataframe to return the column headers of columns where the row has a value of 1. 

##### Then, these column headers are stored into a new column called "HCC". Then we left strip the values where HCC0 is present to get a clean result of just HCC codes as required by the objective of the exercise.

##### Finally we insert "," between each value to seperate them and make it more readably for the user.

In [10]:
hcc_df['HCC'] = hcc_df.dot(hcc_df.columns.str.lstrip('HCC0') + ",")

In [11]:
hcc_df

,HCC01,HCC02,HCC03,HCC04,HCC05,HCC06,HCC07,HCC08,HCC09,HCC10,HCC
MemberID,,,,,,,,,,,
1001,0,0,0,0,0,0,0,0,0,0,
1001,1,0,0,0,1,0,0,1,0,0,"1,5,8,"
1002,0,1,0,0,0,0,1,0,0,0,"2,7,"
1003,0,0,0,0,0,0,0,0,0,0,
1004,1,0,0,0,0,0,0,0,0,0,"1,"
1005,0,0,1,0,0,1,0,0,0,0,"3,6,"


##### Here we do some more cleaning by removing the extra " , " seperator which is assigned at the end of each row value. 

In [12]:
hcc_df['HCC'] = hcc_df['HCC'].str.rstrip(',')

##### Now, we create a summary table by dropping unwanted columns and only keeping the result column ("HCC").

In [13]:
summary_table = hcc_df.drop(hcc_df.columns[:10], axis=1)

##### We add the activity year column to the summary table to make it more insightful

In [14]:
summary_table = pd.concat([summary_table, pd.DataFrame(df['ActivityYear'])], axis=1, sort=False)

In [15]:
summary_table

,HCC,ActivityYear
MemberID,,
1001,,2019
1001,"1,5,8",2020
1002,"2,7",2019
1003,,2020
1004,1,2020
1005,"3,6",2020


##### Finally, we reorder the columns to make the resulting summary table consumable for analytical reporting.

In [16]:
summary_table = summary_table[['ActivityYear', 'HCC']]

In [17]:
summary_table

,ActivityYear,HCC
MemberID,,
1001,2019,
1001,2020,"1,5,8"
1002,2019,"2,7"
1003,2020,
1004,2020,1
1005,2020,"3,6"
